Compare with Tide Gauges
========================

This ({nb-download}`notebook <Compare-Tide-Gauges.ipynb>`) compares model predictions with tide gauge data provided by the [University of Hawaii Sea Level Center](https://uhslc.soest.hawaii.edu/network/), and computes a local solution from the tide gauge data

OTIS format tidal solutions provided by Oregon State University and ESR  
- [http://volkov.oce.orst.edu/tides/region.html](http://volkov.oce.orst.edu/tides/region.html) 
- [https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/](https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/)
- [ftp://ftp.esr.org/pub/datasets/tmd/](ftp://ftp.esr.org/pub/datasets/tmd/)  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  
- [https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models](https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models)

Finite Element Solution (FES) provided by AVISO  
- [https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html)
        
## Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](http://matplotlib.org/)  
 - [cartopy: Python package designed for geospatial data processing](https://scitools.org.uk/cartopy/docs/latest/)  

## Program Dependencies

- `crs.py`: Coordinate Reference System (CRS) routines  
- `io.model.py`: retrieves tide model parameters for named tide models  
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  
- `io.ATLAS.py`: extract tidal harmonic constants from ATLAS netcdf models  
- `io.GOT.py`: extract tidal harmonic constants from GOT tide models  
- `io.FES.py`: extract tidal harmonic constants from FES tide models  
- `solve.py`: estimates the harmonic constants for ocean tides
- `time.py`: utilities for calculating time operations

This notebook uses Jupyter widgets to set parameters for calculating the tidal values.  

## Load modules

In [ ]:
from __future__ import print_function

import netCDF4
import ipywidgets
import numpy as np
import IPython.display
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

# import tide programs
import pyTMD.tools
import pyTMD.compute
import pyTMD.solve
import pyTMD.utilities
import timescale
# autoreload
%load_ext autoreload
%autoreload 2

## Set parameters for program

- Model directory  
- Tide model  

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ocean_elevation())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'GOT4.10'
TMDwidgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
    TMDwidgets.compress,
])

## Find Tide Gauge Data

In [ ]:
# remote directory with tide gauge data
HOST = 'https://uhslc.soest.hawaii.edu/data/netcdf/rqds/global/hourly/'
f = pyTMD.utilities.uhslc_list(HOST, pattern=r'(.*?).nc', sort=True)
# create dropdown with all tide gauge data
TMDwidgets.gauges = ipywidgets.Dropdown(
    options=f,
    value=f[0],
    description='Tide Gauges:',
    disabled=False,
    style=TMDwidgets.style,
)
display(TMDwidgets.gauges)

## Download and Read Tide Gauge Data

In [ ]:
# open tide gauge data
fid = pyTMD.utilities.from_http([HOST,TMDwidgets.gauges.value])
with netCDF4.Dataset(fid.filename, memory=fid.read()) as fileID:
    # read time and station name
    delta_time = fileID.variables['time'][:].squeeze()
    date_string = fileID.variables['time'].units
    station_name, = netCDF4.chartostring(fileID.variables['station_name'][:])
    print(f'{fid.filename}: {station_name}')
    # get station latitude and longitude
    lat, = fileID.variables['lat'][:]
    lon, = fileID.variables['lon'][:]
    # get sea level heights
    sea_level = fileID.variables['sea_level'][:].squeeze()
# reduce to valid points
valid = np.logical_not(sea_level.mask | np.isnan(sea_level))
# convert time
epoch, to_sec = timescale.time.parse_date_string(date_string)
ts = timescale.from_deltatime(delta_time[valid]*to_sec, epoch=epoch)
deltat = ts.tt_ut1
# remove the mean and convert to meters
h = (sea_level.compressed() - sea_level[valid].mean())/1000.0

In [ ]:
%matplotlib widget
# check coordinates on tide grid
f1,ax1 = plt.subplots(num=1, figsize=(8.25,5.25),
    subplot_kw=dict(projection=ccrs.PlateCarree()))
point, = ax1.plot(lon, lat, 'r*', transform=ccrs.PlateCarree())
# add title
ax1.set_title(f'{station_name} ({lat:0.2f}\u00B0N, {lon:0.2f}\u00B0E)')
# add coastlines
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='0.85')
# axis = equal
ax1.set_aspect('equal', adjustable='box')
# no ticks on the x and y axes
ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])
ax1.set_extent([-180, 180, -90, 90])
# stronger linewidth on frame
ax1.spines['geo'].set_linewidth(2.0)
ax1.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
f1.tight_layout()
plt.show()

## Predict tides at measurement times

In [ ]:
# get model parameters
model = pyTMD.io.model(verify=False).elevation(TMDwidgets.model.value)
model.parse_constituents()
# calculate tide elevations
tide = pyTMD.compute.tide_elevations(lon, lat, ts.to_datetime(),
    DIRECTORY=TMDwidgets.directory.value, 
    MODEL=TMDwidgets.model.value, TYPE='time series', 
    EPSG=4326, TIME='datetime', EXTRAPOLATE=True).squeeze()

## Adjust global solution for regional data

In [ ]:
# use constituents from original model
c = model.constituents
amp, ph = pyTMD.solve.constants(ts.tide, h-tide.data, c,
    deltat=deltat, corrections=model.corrections)
# calculate complex phase in radians for Euler's
cph = np.atleast_2d(-1j*ph*np.pi/180.0)
# calculate constituent oscillation
hc = np.ma.array(amp*np.exp(cph))
hc.mask = hc.data == hc.fill_value
stide = pyTMD.predict.time_series(ts.tide, hc, c,
    deltat=deltat, corrections=model.corrections)
stide += tide

## Compare predictions and measured values

In [ ]:
%matplotlib widget
f2, ax2 = plt.subplots(num=2)
ax2.plot(ts.year, h, label='Tide Gauge', lw=2, color='darkorchid')
ax2.plot(ts.year, tide, label='Tide Prediction', color='mediumseagreen')
ax2.plot(ts.year, stide, label='Tide Solution', color='darkorange')
ax2.set_title(f'{station_name} ({lat:0.2f}\u00B0N, {lon:0.2f}\u00B0E)')
lgd = ax2.legend(frameon=False)
lgd.get_frame().set_alpha(1.0)
for line in lgd.get_lines():
    line.set_linewidth(6)
f2.tight_layout()
plt.show()